# 数据集概览
## Load datasets 

In [ ]:
from load import load_all_dataset
X_train, y_train, X_test, y_test = load_all_dataset()
import numpy as np
np.set_printoptions(edgeitems=5,
                    formatter={"float":lambda x: "{:.3f}".format(x)})

## 训练集
### City A (source)

In [ ]:
print("source labeled (City A weak & failure):", X_train.source.shape)
# print(np.where(np.isnan(X_train.source[0])))
print("|- labels:", y_train.source.shape)
print(" |- weak=0:", y_train.source[y_train.source==0].shape)
print(" |- failure=1:", y_train.source[y_train.source==1].shape)
print("source background (City A good):", X_train.source_bkg.shape)

### City B (target)

In [ ]:
print("target labeled (City B weak & failure):", X_train.target.shape)
print("|- labels:", y_train.target.shape)
print(" |- weak=0:", y_train.target[y_train.target==0].shape)
print(" |- failure=1:", y_train.target[y_train.target==1].shape)
print("target background (City B good):", X_train.target_bkg.shape)

## 测试集
### City B (target)

In [ ]:
print("target labeled (City B weak & failure):", X_test.target.shape)
print("|- labels:", y_test.target.shape)
print(" |- weak=0:", y_test.target[y_test.target==0].shape)
print(" |- failure=1:", y_test.target[y_test.target==1].shape)
print("target background (City B good):", X_test.target_bkg.shape)

# Residual Neural Network
## 预处理

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
# 去除NaN
from numpy import newaxis
class FeatureExtractor:

    def __init__(self):
        pass

    def transform(self, X):
        ''' Replace NaN by 0 and flatten the matrix to size (sample, 6720).
        Executed on every input data (i.e., source, bkg, target) and passed
        the resulting arrays to `fit`and `predict` methods in :class: Classifier

        Parameters
        ----------
        `X`: ndarray of (sample, 672, 10)
            3D input dataset(sample, time, features)
        
        Returns
        -------
        `X`: ndarray of (sample, 6720)
            The filtered dataset
        '''
        np.nan_to_num(X, copy=False)
        return X[:,:,:,newaxis]  

fe = FeatureExtractor()
X_train.source = fe.transform(X_train.source)
X_train.target = fe.transform(X_train.target)
X_test.target = fe.transform(X_test.target)
X_train.source.shape

In [ ]:
# 将数据集转换为TensorFlow格式
train_dataset = tf.data.Dataset.from_tensor_slices((X_train.source, y_train.source)).batch(16)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_train.target, y_train.target)).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test.target, y_test.target))

# 额外操作
train_dataset = train_dataset.map( lambda x, y: (tf.image.random_flip_left_right(x), y) )
train_dataset = train_dataset.repeat()
valid_dataset = valid_dataset.repeat()

## 搭建网络模型
参考资料

1. [Introduction to ResNet in TensorFlow 2](https://adventuresinmachinelearning.com/introduction-resnet-tensorflow-2/)

In [ ]:
def res_net_block(filters, conv_size, input_data):
    ''' A residual block of 3 layers
    '''
    # 1st layer with batch normalization
    x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(input_data)
    x = layers.BatchNormalization()(x)
    # 2nd layer with batch normalization, but no activation function
    x = layers.Conv2D(filters, conv_size, activation=None, padding='same')(x)
    x = layers.BatchNormalization()(x)
    # 3rd layer is residual addition with an activation function
    x = layers.Add()([x, input_data])
    x = layers.Activation('relu')(x)
    return x

In [ ]:
# Layer to be used as an entry point into a Network (a graph of layers).
# https://keras.io/api/layers/core_layers/input/
inputs = keras.Input(shape=(672, 10, 1), name="Input_Layer")
# inputs = layers.InputLayer(input_shape=(672, 10), name="Input_Layer")

# 2D convolution layer (e.g. spatial convolution over images).
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
# filters: Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution).
# kernel_size: An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window.
x = layers.Conv2D(32, 2, activation='relu', name="Layer01")(inputs)
x = layers.Conv2D(64, 2, activation='relu', name="Layer02")(x)

# Max pooling operation for 2D spatial data.
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D
# pool_size: integer or tuple of 2 integers, window size over which to take the maximum.
x = layers.MaxPooling2D(2, name="Layer03")(x)

num_res_net_blocks = 10 # 10个ResNet blocks
for i in range(num_res_net_blocks):
    x = res_net_block(64, 2, x)

# [Final layers] a standard CNN layer
x = layers.Conv2D(64, 2, activation='relu', name="Layer-4")(x)
# [Final layers] GAP layer
x = layers.GlobalAveragePooling2D(name="Layer-3")(x)
# [Final layers] dense classification layers
# Just your regular densely-connected NN layer.
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
# units: Positive integer, dimensionality of the output space.
x = layers.Dense(256, activation='relu', name="Layer-2")(x)
# [Final layers] dropout layer
x = layers.Dropout(0.5, name="Layer-1")(x)
# [Final layers] dense classification layers
outputs = layers.Dense(1, activation='softmax', name="Output_Layer")(x)
res_net_model = keras.Model(inputs, outputs)

res_net_model.summary()


## 训练模型
'binary_crossentropy' 效果比 'categorical_crossentropy' 稍好。后者计算的loss直接=0.

In [13]:
import datetime as dt
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./log/{}'.format(
      dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S ResNet")), write_images=True),
  ]
res_net_model.compile(optimizer=keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['acc'])
res_net_model.fit(train_dataset, epochs=100, steps_per_epoch=200,
          validation_data=valid_dataset,
          validation_steps=3, callbacks=callbacks)

Epoch 1/100
200/200 [==============================] - 44s 206ms/step - loss: 28853349122048.0000 - acc: 0.1081 - val_loss: 14470327828480.0000 - val_acc: 0.2917
Epoch 2/100
200/200 [==============================] - 41s 204ms/step - loss: 39866876297216.0000 - acc: 0.0916 - val_loss: 12976597762048.0000 - val_acc: 0.2917
Epoch 3/100
200/200 [==============================] - 40s 201ms/step - loss: 19106625486848.0000 - acc: 0.0797 - val_loss: 11967801589760.0000 - val_acc: 0.2917
Epoch 4/100
200/200 [==============================] - 41s 203ms/step - loss: 35934275895296.0000 - acc: 0.0828 - val_loss: 10949795774464.0000 - val_acc: 0.2917
Epoch 5/100
200/200 [==============================] - 41s 205ms/step - loss: 16959125061632.0000 - acc: 0.0791 - val_loss: 9528608292864.0000 - val_acc: 0.2917
Epoch 6/100
200/200 [==============================] - 41s 205ms/step - loss: 14154781949952.0000 - acc: 0.0744 - val_loss: 9088081592320.0000 - val_acc: 0.2917
Epoch 7/100
200/200 [=========

In [16]:
res_net_model.predict(X_test.target)

array([[1.000],
       [1.000],
       [1.000],
       ...,
       [1.000],
       [1.000],
       [1.000]], dtype=float32)

## 查看Tensorboard

In [14]:
%tensorboard

UsageError: Line magic function `%tensorboard` not found.
